# This runs on google colab, must change runtime type to GPU  

In [1]:
!pip install lightning-bolts["extra"]
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pytorch_lightning as pl
import torch
!apt install libomp-dev
!python -m pip install faiss-gpu
import faiss
from tqdm.notebook import tqdm
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
import time
import gc

     |████████████████████████████████| 282 kB 4.1 MB/s 
     |████████████████████████████████| 1.0 MB 50.3 MB/s 
     |████████████████████████████████| 329 kB 72.3 MB/s 
     |████████████████████████████████| 1.7 MB 76.9 MB/s 
     |████████████████████████████████| 448 kB 70.4 MB/s 
     |████████████████████████████████| 47.6 MB 1.2 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 132 kB 61.8 MB/s 
     |████████████████████████████████| 829 kB 87.9 MB/s 
     |████████████████████████████████| 1.1 MB 43.9 MB/s 
     |████████████████████████████████| 192 kB 61.0 MB/s 
     |████████████████████████████████| 271 kB 70.6 MB/s 
     |████████████████████████████████| 160 kB 68.9 MB/s 
     |████████████████████████████████| 180 kB 70.6 MB/s 
     |████████████████████████████████| 97 kB 7.8 MB/s 
     |████████████████████████████████| 140 kB 56.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for

In [2]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
from google.colab import files
files.upload()

Saving pickled_df.pkl to pickled_df.pkl


{'pickled_df.pkl': b'\x80\x05\x95\xadb\x00\x00\x00\x00\x00\x00\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x0cBlockManager\x94\x93\x94)\x81\x94(]\x94(\x8c\x18pandas.core.indexes.base\x94\x8c\n_new_Index\x94\x93\x94h\x0b\x8c\x05Index\x94\x93\x94}\x94(\x8c\x04data\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x07\x85\x94h\x15\x8c\x05dtype\x94\x93\x94\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94(\x8c\r% no internet\x94\x8c&% internet with broadband subscription\x94\x8c\x07% white\x94\x8c\x07% black\x94\x8c\x07% asian\x94\x8c\x11% native american\x94\x8c\x0bmean income\x94et\x94b\x8c\x04name\x94Nu\x86\x94R\x94h\rh\x0f}\x94(h\x11h\x14h\x17K\x00\x85\x94h\x19\x87\x94R\x94(K\x01MQ\x13\x85\x94h!\x89]\x94(\x8c\x02 1\x94\x8c\x02 2\x94\x8c\

In [6]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
scaler = StandardScaler()

class MyDataModule(pl.LightningDataModule):
    def __init__(self, dataset, labels, batch_size = 64):
        self.dataset = dataset
        self.labels = labels
        self.batch_size = batch_size

    def setup(self):
        X_train, X_test, Y_train, Y_test = train_test_split(self.dataset, self.labels, test_size=0.2, random_state=42)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        X_train, X_test, Y_train, Y_test = torch.tensor(X_train.astype(np.float32), device = 'cuda'), torch.tensor(X_test.astype(np.float32), device = 'cuda'),\
        torch.tensor(Y_train.astype(np.int), device = 'cuda'), torch.tensor(Y_test.astype(np.int), device = 'cuda')
       # X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
        self.training_set = TensorDataset(X_train, Y_train)
        #self.validation_set = TensorDataset(X_valid, Y_valid.ravel())
        self.test_set = TensorDataset(X_test, Y_test)

    def train_dataloader(self):
        train_batch = DataLoader(self.training_set, batch_size=self.batch_size)
        return train_batch

    def val_dataloader(self):
        valid = DataLoader(self.validation_set, batch_size=self.batch_size)
        return valid

    def test_dataloader(self):
        test_batch = DataLoader(self.test_set, batch_size=self.batch_size)
        return test_batch

    def test_epoch_end(self, outputs): 
        results = process_outputs(outputs)
        self.test_results = results
        return results 

class LogisticRegressionModel(pl.LightningModule):
  def __init__(self, input_layer=5, output=10):
    '''method used to define our model parameters'''
    super().__init__()
    self.layer_1 = torch.nn.Linear(input_layer, output)

  def forward(self, x):
    '''method used for inference input -> output'''
    return self.layer_1(x)

  def training_step(self, batch, batch_idx):
      '''needs to return a loss from a single batch'''
      x, y = batch
      logits = self(x)
      #m = torch.nn.LogSoftmax(dim=1)
      loss = torch.nn.CrossEntropyLoss()(logits, y)
      #self.log('train_loss', loss)
      return loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3, weight_decay=.003)
    return optimizer 

In [7]:
# have to scale the data first,lol
def accuracy(model, dataset_dataloader):
  correct = 0
  model.to('cuda')
  model.eval()
  with torch.no_grad():
      for (images, labels) in dataset_dataloader: 
          images = images.to('cuda')
          labels = labels.to('cuda')
          outputs = model(images).to('cuda')
          _, test_predicted = torch.max(outputs.data,1)
          correct += torch.eq(labels, test_predicted).sum()
  accuracy = (correct/len(dataset_dataloader.dataset)).item() * 100 
  return accuracy 

def logistic_regression_classify_mnist(dataset: np.ndarray, labels: np.ndarray):
  device = torch.device("cuda" if torch.cuda.is_available else "cpu")
  dm = MyDataModule(dataset, labels,batch_size=256)
  dm.setup()
  model = LogisticRegressionModel()
  trainer = pl.Trainer(gpus=1, max_epochs = 300, precision=16)
  trainer.fit(model, train_dataloader=dm.train_dataloader())

  return (accuracy(model, dm.train_dataloader()), accuracy(model, dm.test_dataloader()))

In [8]:
!pip3 install pickle5
import pickle5 as pickle
with open("pickled_df.pkl", "rb") as fh:
  data = pickle.load(fh)

     |████████████████████████████████| 132 kB 4.3 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219319 sha256=c94006e0827918a9cd220210272769b712a6450934b0312d0eef59d9dabcce85
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [9]:
data["% no internet"] = data["% no internet"].apply(lambda x: 0 if x <= 10 else(1 if x <= 20 else(2 if x <= 30 else (3 if x <= 40 else(4 if x <= 50 else(5 if x <= 60 else (6 if x <= 70 else(7 if x <= 80 else(8 if x <= 90 else 9)))))))))

In [10]:
dataset = data.drop(columns = ['% no internet', "% internet with broadband subscription"]).to_numpy().astype(np.float32)
labels = data["% no internet"].values.astype(np.float32)

In [11]:
logistic_regression_classify_mnist(dataset, labels)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:733: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  "`trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 60    
-----------------------------------
60        Trainable params
0         Non-trainable params
60        Total params
0.000     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consid

Training: 0it [00:00, ?it/s]

(89.86350297927856, 88.67543339729309)